In [34]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import nltk

In [35]:
data = pd.read_csv("movie_dataset/train.csv")

data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [36]:
data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [37]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True)
val.reset_index(inplace=True)

In [38]:
from konlpy.tag import Okt

okt = Okt()

In [39]:
#train 데이터 전처리
main_pos = []
for sentence in train['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
train['main_pos'] = pd.DataFrame(main_pos)
train.head()

,index,id,document,label,main_pos
0,3474,3475,멜로연기는 최지우가 제일잘하는듯,1,멜로 연기 최지우 제일 잘하는듯
1,221,222,제임스딘 그의 매력을 누가 따라올까...,1,제임스딘 그 매력 누가 따라올까
2,1399,1400,아직도 그립다...폴워커...,1,아직도 그립다 폴 워커
3,1562,1563,별반개도 완전 아까움영화관서 보다 배멀미했던 기억,0,반개 완전 아까 움 영화관 보다 배 멀미 했던 기억
4,3605,3606,좋은 강연을 해주서서 고맙습니다,1,좋은 강연 해주 서서 고맙습니다


In [40]:
X_train = train.main_pos
y_train = train.label

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)

In [42]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X_train_vec, y_train)

MLPClassifier()

In [43]:
#val 데이터 전처리

main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val['main_pos'] = pd.DataFrame(main_pos)

val

,index,id,document,label,main_pos
0,1128,1129,선가(禪家)에 관한 영화 중 가장 개성있는 작품,1,선 관 영화 중 가장 개성 있는 작품
1,2960,2961,쓰레기더ㅏㅋㅋㅋㅋㅋㅋㅋㅋㅋ돈주고봣엇는데내돈돌려내,0,쓰레기 더 돈 주고 봣 는데 돈 돌려내
2,4173,4174,만화가 10000000000000000배 훨씬 나아,0,만화가 배 훨씬 나아
3,1565,1566,동화같은게 이세상에 존재하지않아.,1,동화 같은게 세상 존재 하지 않아
4,2170,2171,액션 하나로 승부보기엔 너무 엉맘징창인 영화 ㅡ.ㅡ,0,액션 하나로 승부 보기 너무 맘징창 영화
...,...,...,...,...,...
1245,1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1,히치콕 이 영화 봤다면 분명 박수 쳤을듯
1246,2922,2923,엠비씨 하얀거탑 골든타임은 어디가고 드라마제목부터유치한 메디컬탑팀을만들었나,0,엠비씨 하얀거탑 골든타임 어디가 드라마 제목 유치한 메디컬탑팀 만들었나
1247,1394,1395,천국에서도 부잣집이 이렇게는 안하겠다,0,천국 부잣집 이렇게는 하겠다
1248,165,166,이렇게 원작을 망치기도 힘들겠다ㅋ,0,이렇게 원작 망치 기도 힘들겠다


In [44]:
X_val = val.main_pos
y_val = val.label

In [45]:
X_val_vec = vectorizer.transform(X_val)

In [46]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[1 0 1 ... 0 0 0]


In [47]:
from sklearn import metrics

print("accuracy=", metrics.accuracy_score(y_val, y_pred))

accuracy= 0.82


In [48]:
test = pd.read_csv('movie_dataset/test.csv')

In [49]:
test.isnull().sum()

id          0
document    0
dtype: int64

In [57]:
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test['main_pos'] = pd.DataFrame(main_pos)

test

,id,document,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동 적이네요 펑펑 울었습니다
...,...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화,계 제대로 안되고 뭔가 부족한 느낌 들고 찝찝한 영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....,내생 최악 영화 하나 졸려서 미칠 뻔
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!,지금 나온 드라마 중 최고 였습니다 점 만점
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..,영화 값 엄청난 공연 보았다 이 웬 횡재


In [54]:
submission = pd.read_csv('movie_dataset/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [55]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 1 ... 1 1 1]


In [56]:
submission['label'] = pred_test

submission

,id,label
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,1


In [26]:
submission.to_csv("submission_mycode_MLP.csv", index=False)